In [96]:
from datasets import load_dataset
import polars as pl
from huggingface_hub import snapshot_download
import os
from typing import Sequence
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam
from datasets import Dataset
from pathlib import Path
from pprint import pprint
from IPython.display import Markdown, display

In [38]:
for dataset_name in [
    # "lara-martin/FIREBALL",
    # "MinervaAI/Aesir-Preview",
    # "hieunguyenminh/roleplay",
    # "chargoddard/rpguild",
    # "jondurbin/cinematika-v0.1",
    # "codeparrot/apps",
    # "glaiveai/glaive-code-assistant-v3",
    # "jondurbin/py-dpo-v0.1",
    # "lemonilia/roleplaying-forums-raw",
    # "deepmind/code_contests",
    # "SenseLLM/ReflectionSeq-DS",
    # "openai/openai_humaneval",
    # "argilla/distilabel-intel-orca-dpo-pairs"
    # not downloaded
    # Squish42/bluemoon-fandom-1-1-rp-cleaned
]:
    # run this fn to get the local directory of the dataset
    snapshot_download(dataset_name, repo_type="dataset")

In [ ]:
files = snapshot_download("lemonilia/roleplaying-forums-raw", repo_type="dataset")
files

In [ ]:
dataset = load_dataset("PygmalionAI/PIPPA", trust_remote_code=True)['train']
dataset_pl = dataset.to_polars()


def format_conversation_oai(sample: dict):
    # TODO handle interpolation and character intros
    conversation = sample["conversation"]
    description = sample["bot_description"]
    out_conv: Sequence[ChatCompletionMessageParam] = [
        {"role": "assistant", "content": description}
    ]
    for msg, is_human in zip(conversation["message"], conversation["is_human"]):
        out_conv.append(
            {"role": "user" if is_human else "assistant", "content": msg}
        )
    print(out_conv)
    return out_conv


for sample in dataset_pl.iter_rows(named=True):
    print(sample.keys())
    format_conversation_oai(sample)
    break

In [ ]:
cinematika_root_dir = snapshot_download("jondurbin/cinematika-v0.1", repo_type="dataset")
os.listdir(cinematika_root_dir)

In [ ]:
for filename in os.listdir(cinematika_root_dir):
    file_path = os.path.join(cinematika_root_dir, filename)
    _, extension = os.path.splitext(file_path)
    print(filename)
    if extension == ".parquet":
        dataset = Dataset.from_parquet(file_path)
        print(len(dataset))

In [78]:
scene_by_scene_dataset = Dataset.from_parquet(os.path.join(cinematika_root_dir, "scene_by_scene.parquet"))
actions_dataset = Dataset.from_parquet(os.path.join(cinematika_root_dir, "actions.parquet"))
plain_scenes_dataset = Dataset.from_parquet(os.path.join(cinematika_root_dir, "full_script.parquet"))

In [ ]:
rp_root_dir = snapshot_download("chargoddard/rpguild", repo_type="dataset")
print(rp_root_dir)
os.listdir(rp_root_dir)
print(open(os.path.join(rp_root_dir, "README.md")).read())

In [90]:
rp_grammar_filtered_path = os.path.join(rp_root_dir, "grammar_filtered", "train-00000-of-00001.parquet")
rp_grammar_filtered_dataset = Dataset.from_parquet(rp_grammar_filtered_path)

In [108]:
rp_grammar_filtered_dataset

Dataset({
    features: ['username', 'char_name', 'bio', 'context', 'reply', 'char_confidence'],
    num_rows: 27053
})

In [122]:

from typing import Dict, List
def dictl(dict_of_lists: Dict[str, List]) -> Sequence[dict]:
    """
    Dict of lists to list of dicts.
    """
    return [dict(zip(dict_of_lists.keys(), t)) for t in zip(*dict_of_lists.values())]

batch_size = 4
for batch in rp_grammar_filtered_dataset.iter(batch_size=batch_size):
    samples = dictl(batch)
    for i, sample in enumerate(samples):
        display(Markdown(f"### Sample {i}"))
        for msg in sample['context']:
            display(Markdown(f"**{msg['char_name']}**\n\n {msg['text']}"))
    break

### Sample 0

**Rithrynn Syn'Dial**

 "Oh, no; no! Everything's fine!" Rithrynn quickly replied to Sharon's concerned expression; eluding to the fact that Liaena had left the house; and swiftly conjuring up the most reassuring smile that she could give him. Leaning a little over him to peer into the room, noticing that it was empty; Rithrynn's eyes fell back down to Sharon.

"Mind if I come in?"

Rithrynn asked, yet the Night Elf had already squeezed herself through the opening in the door and inside the room; pushing Sharon back inside as she entered before the High Elf could even have time to answer properly.

Standing in the room, the Sentinel took a moment to gaze around in the dimly lit room. Given the Night Elves' natural affinity for darkness and superior night vision, the lack of lighting bothered her much less so than what it probably did the High Elf.
Moving one leg behind the other, she used the back of her heel to close the door shut behind her. As they were finally alone, separated and removed from the noise in the rest of the house; Rithrynn's attention finally returned to Sharon. Walking over to the High Elf, Rithrynn grabbed him by his shoulders as she looked down at him with a look of almost unbridled glee as she smiled.

"I just wanted to talk to you, Shaw... "

**Sharon Daerin'Lei**

 Sharon gazed back up at Rithrynn for a couple of moments, his eyes racing over happy features on her face.

Although his old, natural instincts shouted out the obvious warning signs; the High Elf slowly nodded, before smiling back at Rithrynn.

"Of course," Sharon confirmed, making an attempt to move; yet Rithrynn's solid grip prevented him from doing much. Staring back at the Night Elf, Sharon eventually motioned towards a set of sofas and table at the side of the room.
"Why don't you find yourself a seat over there, and I will get some light up in this room? I really did not expect anyone at this hour."

---

"If you hadn't been so damnable lucky with the appearance of that elf, you'd have been MINE!" Sounded Astrid's angry voice from the bag as it shuffled around, much to Donald's discomfort as he was on the bottom.

Doin furrowed his brow at the bag, intrigued by the unusual sight. There was a lot he did not know about Liaena, yet he learned something new everything he met her.

"Say'e, there shouldn't be annea problem with ye spendin' a night over. We got more rooms than we can fill, and we usually don't rent them out to just anybody; this being guild property and all." Doin finally said, giving Liaena a smile as he picked up his pipe.
"After all, the boss owe me some favors. I'll get ye tha finest room we have available!"

"Hopefully without any overgrown fairies breaking in through the windows this time." Frarn mentioned jokingly as he walked up next to his father. However, the two dwarves paused as their eyes drifted over to Kathlin, in Liaena's form; who was currently standing in the room.
"Is that a friend of yours?"

Doin peered over to Kathlin, before chuckling.

"I'm pretty sure she is, son. After all, who in their right mind would ever trust their own image in the hands of someone they didn't trust absolutely? Hah!"

**Sharon Daerin'Lei**

 Sharon gazed back up at Rithrynn for a couple of moments, his eyes racing over happy features on her face.

Although his old, natural instincts shouted out the obvious warning signs; the High Elf slowly nodded, before smiling back at Rithrynn.

"Of course," Sharon confirmed, making an attempt to move; yet Rithrynn's solid grip prevented him from doing much. Staring back at the Night Elf, Sharon eventually motioned towards a set of sofas and table at the side of the room.
"Why don't you find yourself a seat over there, and I will get some light up in this room? I really did not expect anyone at this hour."

---

"If you hadn't been so damnable lucky with the appearance of that elf, you'd have been MINE!" Sounded Astrid's angry voice from the bag as it shuffled around, much to Donald's discomfort as he was on the bottom.

Doin furrowed his brows at the bag, intrigued by the unusual sight. There was a lot he did not know about Liaena, yet he learned something new every time he met her.

"Say'e, there shouldn't be annea problem with ye spendin' a night over. We got more rooms than we can fill, and we usually don't rent them out to just anybody; this being guild property and all." Doin finally said, giving Liaena a smile as he picked up his pipe.
"After all, the boss owe me some favors. I'll get ye tha finest room we have available!"

"Hopefully without any overgrown fairies breaking in through the windows this time." Frarn mentioned jokingly as he walked up next to his father. However, the two dwarves paused as their eyes drifted over to Kathlin, in Liaena's form; who was currently standing in the room.
"Is that a friend of yours?"

Doin peered over to Kathlin, before chuckling.

"I'm pretty sure she is, son. After all, who in their right mind would ever trust their own image in the hands of someone they didn't trust absolutely? Hah!"

**Rithrynn Syn'Dial**

 Rithrynn continued to peer back at Sharon for a couple of moments after he asked her. With a soft smile, she subtly nodded with her head as she removed her hands from him.

"Okay... " The Sentinel spoke, before stepping over to the nearby sofa. It was a rare thing to have Liaena away and Sharon still in the house, seeing as the nosy human usually didn't let Sharon get far away from her notice. In one sense, the attack from the mysterious assassins was a good thing as it finally took Liaena's paranoia away from Rithrynn and over to them.

As she walked over to the sofa, she brushed off some of the snow flakes that still remained on her cloak from the outside, before extending her butt and slumping it down into the cushioning embrace of the sofa.

"Have you been up here by yourself for a long time?" Rithrynn asked as she had become comfortable in the couch, allowing her eyes to trail back to the High Elf in her room.
"I hope the assassins are not forcing you into solitude... "

Rithrynn said as she gave Sharon a sympathetic look through the room, while fiddling with her gloves in the meantime as she removed them and placed them, alongside her Sentinel bracers, onto the table before her.

...

**Meirie Leran**

 "However you wish, miss Vashi..." Mei had given up on trying to argue already and was simply following the night elf mage. The very idea of an arcane wielding night elf seemed absurd at first. Mei herself had spend some time at the lands of the night elves years ago when the relationships between their two nations were forming so she got a chance to study their libraries and ancient texts... at least withing the allowed range without trying to read secret documents and the like. What she did learn was that night elves abandoned anything arcane related thousands of years ago... why this night elf used magic like this was quite the mystery for the draenei warrior.

 "Will not be surprised... " Meirie added at the statement of possible secret laboratories down in the basement somewhere. It was the perfect place for it really, underground so the noise from it would be deafened, no flashes of light to alert the surroundings and very well protected in case... something blows up.

 "Don't worry miss Vashi... from now on I will protect you and your things. If someone wants to take something or touch something, I will give them a stern scolding." She said with a smile, wondering exactly how big was the basement of this mansion. She knew that they were big for she had entered one of the other estates before for work. Had to clean a basement from some hostile beasts... how they ended there she did not ask for the pay was good back then.

 "Miss... I've been meaning to ask, but how did you end in a night elven warband?" The warrior asked eventually." I mean a woman with your abilities would surely have other prospects?"

**Sharon Daerin'Lei**

 "Wow. You might need to restock these, Sharon"

A low but still very noticeable mental groan could almost be felt emanating from the smaller High Elf on Liaena's shoulder as she entered into the (now posthumously) room o jam.

Sharon said nothing, but merely gazed at the rows upon rows of empty or missing bottles of jam, half of which lay scattered across the floor in an unceremonious fashion. Indeed, the Night Elves had apparently not afforded even the jam any more mercy than they had the wine storage by the looks of it! The mage could only dread the number at the end of the bill once everything had been refilled.

Perhaps it might even be better just to let Friedrick burn said bill before he could even see it.

"It's the antidote. Take it. I don't want to mislay you down here"

"I think there are worse places you could mislay me. And you don't have to leave the house to find them." Sharon said half-jokingly, half concerned as he threw a quick glance around before he accepted the potion.

Moments later, he would stand back at his full, very-much-so normal and not-pocket-sized stature. Taking a moment to tidy up his deep azure blue robe and adjust the mithril chestplate under it so that it sat properly, the blond haired High Elf would step over to where Liaena was working not far from the unconscious body of the large Death Knight. As he came close enough, he would extend his staff wielding arm and a bit of his cloak as well as he wrapped it around Liaena as she was working, before speaking as he looked towards her.

"I think you did really well today, Liaena. My brother has made this life anything but easy... no matter how much you may make it seem the exact opposite sometimes. I really appreciate having you by my side, Liaena." He smiled, simply taking a moment where he simply looked at her.

That attention however would be broken by a low groan emanating from the now half-sleeping Draenei as she shuffled somewhat around on the floor, slowly recovering from her sleep.

Casting a look between Liaena and Malizia, Sharon would reach into his arcane pouch hanging at his side with his other hand to produce a pair of handcuffs made out of Thorium, with some mystical magical runes written into them. 

"I got some anti-mage cuffs that could prevent her from casting any spells... " Sharon explained as he held them up in front of Liaena with a slight grin, before suffering the inevitably stare that any man outside of law enforcement would receive when casually revealing himself to be carrying handcuffs.
"You never know when they might come in handy... totally not a souvenir from my academy time in Dalaran, no... "

...

"Anyhow!" Brushing past the awkward silence, he'd look back at Liaena.
"They would block her spells, but I don't think you'd be able to shrink them. So we are going to have to make a choice, whether we want to contend with a shackled 8ft tall martial monstrosity of death - literally -, or a shrunk adept student of some of the darkest magic known to mortals and deities alike."

Nodding, he'd gaze slightly away.

"Just like... whichever option you'd feel the most comfortable with."

### Sample 3

**Rithrynn Syn'Dial**

 "AAAAAAAAAAAHHH!"

"OWWWWH!!"

*Crash*
*BOOM*

"Let me introduce you to my little friend, GRAVITY, BITCH!"

"AIII-"
*DROOOHHHMM!!*

Serphia's support team/goon squad had long since given up any attempt at interfering with the fight of the two, giant behemoth currently fighting over the piece of dirt that had once been Sharon's garden! Instead, they had taken up a (relatively) safe distance in the surrounding streets and simply gazed up at the ash cloud covered silhouettes of Serphia and Malizia as the two furious women duked it out in a no-holds-barred beatdown of each other!

In one scene, Malizia charged forward, launching her own head right into Serphia's chest and knocking the gigantic Sentinel Captain into the mansion, promptly making her serve as an oversized wrecking ball that immediately demolished the roof of the east wing!

"Giving up yet, you little troll?"

"You are going to DIE for that!!"

Serphia retaliated by swiping the feet(hoofs) out from under Malizia, the giant red draenei-demon-thingy falling to the ground with a deep rumble! Malizia answered by jumping onto Serphia, only for Serphia to pull one leg back, place her feet up under Malizia's chest, and push back, causing Malizia to go flying up into the air with a scream!

Back at the mansion ground, Sentinels in various states of clothes or stripping could be seen running out of the front door of the mansion, gathering in a small crowd outside in response to the growing fire.

There was an awkward pause among the dozens of drunk and half-sober sentinels that had gathered outside of the mansion as they simply stared dumbfounded at the fire spreading in the second floor, before one of them suddenly stepped forward and raised her stolen mug.

"BEST PARTY EVER!!"

In response to the sentinel, the rest of her sisters raised their fists and cried out as well, jumping up and down with their drinks as they cheered on the burning mansion!
In the background, angry nightsabers could be seen jumping out of windows of the mansion to escape the fire, before running amok in the streets!

"WHOOOO-HOOOO!!"
"YEAAH!!"

...

"So... where do we go now?"

**Sharon Daerin'Lei**

 "So are we leaving? I like this plan if you do, Mister Dal'dieb"

The High Elf's gaze passed from the curious, strangely optimistic looking Kathlin over to Liaena, then back again.

"If I said no, I doubt I'd get to the o in that word before Liaena had already dragged me off to Westfall. I'm still not sure if giving her that haste buff was a good idea!"

Sharon privately thought, also unsure whether he should also mention the potential of the spell having taking 1 or 2 years off her life expectancy.

"Come to think of it, I have always wanted a new place closer to the beach... " Sharon muttered, moving one hand up to his chin in a thoughtful expression, before he turned to Liaena and Kathlin, and nodded; all the while instinctively bracing himself in case Liaena should decide to run somewhere with him along!

"Not to speak ill of your skills or anything, but I do believe the barrier I set up is more than a match for a yet so simple apprentice." Sharon said as he glanced over at Kathlin. Though Kathlin had never shown anything but t e best intentions and did seem to have an elevated magical potential despite her simple class, Sharon doubted that her skills were enough to contest with that of a long lived High Elf and esteemed student and teacher at the Dalaran academy of magical sorcery!

"Aran kehn, rahs fuhs." Sharon suddenly muttered, before thumping the butt of his silver staff into the ground two times. 

For a moment, a creeping silence ran across the entire mansion like a wave, as the invisible enchantment was dispelled by the utterance of the keyphrase. The only noise was the not-so-subtle noise of the fleeing elves and nightsabers who apparently already had decided it was time to ditch his soon-to-be ex-mansion!

"One of the first things we were taught at the Institute of Enchantment back at Dalaran was to attach a keyphrase, or password if you will, to our *simpler* wards, so that we could more easily unravel them at a moment's notice than have to go through a lengthy ritual of disenchantment." 

Sharon explained with a smile as he looked over at Kathlin, apparently enjoying any chance he got to show off his superior magical knowledge!

"Now, I will begin the ritual of teleporta,- thank you, Friedrick." 

Sharon interrupted himself to thank Friedrick as the butler seemingly appeared out of nowhere and handed him a Rune of Teleportation, before the High Elf returned to focusing on his ritual.

"This is going to take a few seconds, a minute max... any special destination in mind?" He asked, turning to look over at Liaena with a smile, all the while motioning for them to come closer. However, his confident smile would suddenly fade for a moment, as if the High Elf had just remembered about something.

Turning towards Liaena, he quirked an eyebrow at her.

"Liaena... aren't we forgetting about someone?"